<a href="https://colab.research.google.com/github/NAVEED261/MY-AI-ASSISTANT/blob/main/successfuly_deploying_of_local_to_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_google_genai langgraph_sdk

In [2]:
from google.colab import userdata

%env GOOGLE_API_KEY = {userdata.get('GEMINI_API_KEY')}

import os
print(os.environ["GOOGLE_API_KEY"])

env: GOOGLE_API_KEY={userdata.get('GEMINI_API_KEY')}
{userdata.get('GEMINI_API_KEY')}


In [3]:
%env LANGCHAIN_API_KEY = {userdata.get('LANGCHAIN_API_KEY')}
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-academy"

env: LANGCHAIN_API_KEY={userdata.get('LANGCHAIN_API_KEY')}


In [8]:
# from langgraph_sdk import get_client
from langgraph_sdk import get_client
# The URL provided below is invalid, please provide a valid url.
URL = 'https://9f15-2404-3100-140f-7e04-317e-300c-d6d1-5095.ngrok-free.app'
client = get_client(url = URL)
assistants = await client.assistants.search()

In [9]:
from langchain_core.messages import HumanMessage, convert_to_messages
thread = await client.threads.create()
input_messages = HumanMessage(content="Multiply 4  and 9")
async for event in client.runs.stream(thread["thread_id"],assistant_id="agent",input = {"messages":[input_messages]},stream_mode="values"):
  print(event)

StreamPart(event='metadata', data={'run_id': '1efcd9b8-9a2e-624d-a17e-025089d3b290', 'attempt': 1})
StreamPart(event='values', data={'messages': [{'content': 'Multiply 4  and 9', 'additional_kwargs': {'example': False, 'additional_kwargs': {}, 'response_metadata': {}}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '4f2c07d3-540f-405a-9cc7-4d9516db35a7', 'example': False}]})
StreamPart(event='values', data={'messages': [{'content': 'Multiply 4  and 9', 'additional_kwargs': {'example': False, 'additional_kwargs': {}, 'response_metadata': {}}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '4f2c07d3-540f-405a-9cc7-4d9516db35a7', 'example': False}, {'content': '', 'additional_kwargs': {'function_call': {'name': 'multiply', 'arguments': '{"a": 4.0, "b": 9.0}'}}, 'response_metadata': {'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, 'type': 'ai', 'name': None, 'id': 'run-9b0b03f8-3fbb-42da-abad-b63eee

In [10]:
from langchain_core.messages import HumanMessage, convert_to_messages
thread = await client.threads.create()
input_messages = HumanMessage(content="Multiply 4  and 9")
async for event in client.runs.stream(thread["thread_id"],assistant_id="agent",input = {"messages":[input_messages]},stream_mode="messages"):
  print(event.event)

metadata
messages/metadata
messages/partial
messages/partial
messages/metadata
messages/complete
messages/metadata
messages/partial
messages/partial


In [16]:
from langchain_core.messages import HumanMessage, convert_to_messages
thread = await client.threads.create()
input_messages = HumanMessage(content="Multiply 4  and 9")
async for event in client.runs.stream(thread["thread_id"],assistant_id="agent",input = {"messages":[input_messages]},stream_mode="values"):
   messages = event.data.get('messages',None)
   if messages:
        print(convert_to_messages(messages)[-1])
   print('='*25)

content='Multiply 4  and 9' additional_kwargs={'additional_kwargs': {'example': False, 'additional_kwargs': {}, 'response_metadata': {}}, 'response_metadata': {}, 'example': False} response_metadata={} id='7389c25f-2def-44b9-996c-feab7b2d055c'
content='' additional_kwargs={'additional_kwargs': {'function_call': {'name': 'multiply', 'arguments': '{"a": 4.0, "b": 9.0}'}}, 'response_metadata': {'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, 'example': False, 'invalid_tool_calls': [], 'usage_metadata': {'input_tokens': 169, 'output_tokens': 3, 'total_tokens': 172, 'input_token_details': {'cache_read': 0}}} response_metadata={} id='run-38918e6b-1b86-4e48-95c3-555143b0eecc-0' tool_calls=[{'name': 'multiply', 'args': {'a': 4.0, 'b': 9.0}, 'id': '470ac40d-9f79-4aa9-a05c-a0c0af230d11', 'type': 'tool_call'}]
content='36' name='multiply' id='983df7a1-6b4e-4e1f-b115-8d38fd830305' tool_call_id='470ac40d-9f79-4aa9-a05c-a0c0af230d11'
cont

In [17]:
thread = await client.threads.create()
input_message = HumanMessage(content="Multiply 2 and 3")

def format_tool_calls(tool_calls):
    """
    Format a list of tool calls into a readable string.

    Args:
        tool_calls (list): A list of dictionaries, each representing a tool call.
            Each dictionary should have 'id', 'name', and 'args' keys.

    Returns:
        str: A formatted string of tool calls, or "No tool calls" if the list is empty.

    """

    if tool_calls:
        formatted_calls = []
        for call in tool_calls:
            formatted_calls.append(
                f"Tool Call ID: {call['id']}, Function: {call['name']}, Arguments: {call['args']}"
            )
        return "\n".join(formatted_calls)
    return "No tool calls"

async for event in client.runs.stream(
    thread["thread_id"],
    assistant_id="agent",
    input={"messages": [input_message]},
    stream_mode="messages",):

    # Handle metadata events
    if event.event == "metadata":
        print(f"Metadata: Run ID - {event.data['run_id']}")
        print("-" * 50)

    # Handle partial message events
    elif event.event == "messages/partial":
        for data_item in event.data:
            # Process user messages
            if "role" in data_item and data_item["role"] == "user":
                print(f"Human: {data_item['content']}")
            else:
                # Extract relevant data from the event
                tool_calls = data_item.get("tool_calls", [])
                invalid_tool_calls = data_item.get("invalid_tool_calls", [])
                content = data_item.get("content", "")
                response_metadata = data_item.get("response_metadata", {})

                if content:
                    print(f"AI: {content}")

                if tool_calls:
                    print("Tool Calls:")
                    print(format_tool_calls(tool_calls))

                if invalid_tool_calls:
                    print("Invalid Tool Calls:")
                    print(format_tool_calls(invalid_tool_calls))

                if response_metadata:
                    finish_reason = response_metadata.get("finish_reason", "N/A")
                    print(f"Response Metadata: Finish Reason - {finish_reason}")

        print("-" * 50)

Metadata: Run ID - 1efcd9c4-a74a-6f13-aa4e-0900ddc68141
--------------------------------------------------
Tool Calls:
Tool Call ID: 3f57722c-d872-4da2-93e6-73727fe5cd42, Function: multiply, Arguments: {'a': 2.0, 'b': 3.0}
Response Metadata: Finish Reason - N/A
--------------------------------------------------
Tool Calls:
Tool Call ID: 3f57722c-d872-4da2-93e6-73727fe5cd42, Function: multiply, Arguments: {'a': 2.0, 'b': 3.0}
Response Metadata: Finish Reason - STOP
--------------------------------------------------
AI: The
Response Metadata: Finish Reason - N/A
--------------------------------------------------
AI: The result is 6.
Response Metadata: Finish Reason - STOP
--------------------------------------------------
